# Imports

In [ ]:
!pip install -qq -U diffusers==0.11.1 transformers ftfy gradio accelerate

In [ ]:
import inspect
from typing import List, Optional, Union

import numpy as np
import torch

import PIL
import gradio as gr
from diffusers import StableDiffusionInpaintPipeline

In [ ]:
device = "cuda"
model_path = "runwayml/stable-diffusion-inpainting"

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
).to(device)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


# Clipscore import

In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torchmetrics[multimodal]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers>=4.0

In [ ]:
from torchvision import transforms
from PIL import Image
import torch
from torchmetrics.functional.multimodal.clip_score import clip_score

# data

In [ ]:
re_prompts = ["tiger", "window", "cat", "stack of books", "squirrel",
           "black and white mickey mouse","metal dustbin", "fighter jet", "house","dog", "gray computer",
           "pink tulip","monkey","apple", "white suitcase", "tv", "omelette", "cabybara",
           "oranges", "cookies", "dog", "pool", "stack of books", "magic wand", "pile of rocks"]

full_prompts = ["an tiger walking through a forested area",
              "a window that is next to an eagle statue",
              "a large cat sitting and waiting for a treat",
              "a stack of books sitting on top of a table",
              "a squirrel sitting in an evergreen tree",
              "a black and white mickey mouse jumping a skate board in the air",
              "a metal dustbin on the side of a street",
              "an fighter jet parked in a spot at the airport",
              "A house that is sitting on a runway",
              "a dog entering a crosswalk along a city street",
              "a gray computer sitting on the floor ",
              "a vase sitting in the sun holding a pink tulip",
              "a monkey opening the doors from a bathroom",
              "a vase of apple on a wood table",
              "a kitchen with a white suitacase oven sitting on top of a counter",
              "a tv on top of a stove",
              "a bowl of soup sitting on a plate with omelette and spoon near a basket of crackers",
              "a gray cabybara lying on a carpet in a room",
              "two bagels next to three oranges on a white plate",
              "a couple of cookies sitting on a blue plate on a computer desk",
              "a dog sits on a small bed in a small room",
              "a pool is on the floor on wooden slats",
              "there is a stack of books laid out in front of a couch",
              "a girl standing up with an arm outstretched and a magic wand in hand",
              "a laptop sitting on a pile of rocks next to a remote control"]

total_samples = 25

# Genearate output from mask and input image and prompt

In [ ]:
def inpainting(mask_image, image, prompt):
  mask_image = mask_image.resize((512, 512))
  image = image.resize((512, 512))
  guidance_scale=7
  # prompt = "car"
  num_samples = 3
  generator = torch.Generator(device="cuda").manual_seed(0) # change the seed to get different results
  images = pipe(
      prompt=prompt,
      image=image,
      mask_image=mask_image,
      guidance_scale=guidance_scale,
      generator=generator,
      num_images_per_prompt=num_samples,
  ).images

  # insert initial image in the list so we can compare side by side
  return images

In [ ]:
def calculate_clipscore(images, caption):
  total_score = 0
  for image in images:
    convert_tensor = transforms.ToTensor()
    img_tensor = convert_tensor(image)
    score = clip_score(img_tensor, caption, "openai/clip-vit-base-patch16")
    print(score.detach())
    total_score+= score.detach().item()
  #print("avg: ", total_score/len(images))
  return total_score/len(images)

## Original Masks

In [ ]:
original_clipscore = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "input_mask_"+str(i)+".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  original_clipscore.append(curr_clipscore)

  # clean variables
  del output_gen

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, curr_clipscore))


# save data to file
np.save("original_clipscores.npy", np.array(original_clipscore))


  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(21.7584)
tensor(21.9757)
tensor(21.9757)
current progress: 1/10, 21.903286


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(23.4928)
tensor(24.0820)
tensor(24.3890)
current progress: 2/10, 23.987937


...

tensor(21.2288)
tensor(21.3560)
tensor(20.2383)
current progress: 8/10, 20.941010


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(21.2093)
tensor(20.5985)
tensor(20.6993)
current progress: 9/10, 20.835696


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(19.7509)
tensor(19.5826)
tensor(19.4625)
current progress: 10/10, 19.598688


## Noise Masks 

in batches: each batch: mode, noise level for all images

In [ ]:
original_clipscore = np.load("original_clipscores.npy")

In [ ]:
# 0, 1, 2
mode = 0 
# 0, 1 ,2, 3
noise_level = 2


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(22.0874)
tensor(21.9767)
tensor(21.9757)
current progress: 1/10, -0.005020


  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


...

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(20.4834)
tensor(20.4562)
tensor(20.4883)
current progress: 9/10, 0.017266


  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(18.2347)
tensor(18.2347)
tensor(19.5044)
current progress: 10/10, 0.048001
0.0020439882975234196


In [ ]:
# 0, 1, 2
mode = 0 
# 0, 1 ,2, 3
noise_level = 3


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(21.9757)
tensor(21.9501)
tensor(21.9509)
current progress: 1/10, -0.002540


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(24.4200)
tensor(24.4191)
tensor(24.4294)
current progress: 2/10, -0.018129


...

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(20.4562)
tensor(20.4562)
tensor(20.4562)
current progress: 9/10, 0.018213


  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(18.2347)
tensor(18.2546)
tensor(18.2077)
current progress: 10/10, 0.069718
0.003239017583422904


In [ ]:
# 0, 1, 2
mode = 1 
# 0, 1 ,2, 3
noise_level = 0


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(21.7716)
tensor(21.9757)
tensor(22.0042)
current progress: 1/10, -0.000634


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(23.4834)
tensor(23.7154)
tensor(23.8841)
current progress: 2/10, 0.012242


...

tensor(21.2253)
tensor(21.2899)
tensor(21.1270)
current progress: 8/10, -0.013038


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(21.0256)
tensor(20.7945)
tensor(20.7049)
current progress: 9/10, -0.000287


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(19.7800)
tensor(19.5549)
tensor(19.3706)
current progress: 10/10, 0.001541
-0.00016620392101671474


In [ ]:
# 0, 1, 2
mode = 1 
# 0, 1 ,2, 3
noise_level = 1


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(21.9757)
tensor(21.9415)
tensor(21.9757)
current progress: 1/10, -0.002786


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(23.2330)
tensor(23.7780)
tensor(23.9446)
current progress: 2/10, 0.014010


...

tensor(21.2187)
tensor(21.4269)
tensor(21.2643)
current progress: 8/10, -0.017301


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(21.2244)
tensor(21.1304)
tensor(20.7264)
current progress: 9/10, -0.009184


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(19.6922)
tensor(19.4517)
tensor(19.3293)
current progress: 10/10, 0.005489
-0.000827370530272015


In [ ]:
# 0, 1, 2
mode = 1 
# 0, 1 ,2, 3
noise_level = 2


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(21.6952)
tensor(21.9757)
tensor(21.9757)
current progress: 1/10, 0.000962


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(23.1066)
tensor(24.3829)
tensor(24.3461)
current progress: 2/10, 0.001781


...

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(20.4562)
tensor(21.3034)
tensor(20.4562)
current progress: 9/10, 0.004660


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(18.6864)
tensor(19.5349)
tensor(19.5196)
current progress: 10/10, 0.017944
0.002877195694689189


In [ ]:
# 0, 1, 2
mode = 1 
# 0, 1 ,2, 3
noise_level = 3


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

tensor(21.5452)
tensor(21.9596)
tensor(21.9425)
current progress: 1/10, 0.003996


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(24.3803)
tensor(24.3821)
tensor(24.3828)
current progress: 2/10, -0.016417


  0%|          | 0/50 [00:00<?, ?it/s]

...

tensor(21.2848)
tensor(20.3987)
tensor(20.6372)
current progress: 8/10, 0.007996


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(20.4538)
tensor(20.6407)
tensor(21.1737)
current progress: 9/10, 0.003821


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(18.7360)
tensor(19.3630)
tensor(19.7788)
current progress: 10/10, 0.015616
0.001641630994957688


In [ ]:
# 0, 1, 2
mode = 2 
# 0, 1 ,2, 3
noise_level = 0


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

tensor(21.7449)
tensor(21.9474)
tensor(21.9938)
current progress: 1/10, 0.000363


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(23.6214)
tensor(22.8196)
tensor(24.3522)
current progress: 2/10, 0.016266


  0%|          | 0/50 [00:00<?, ?it/s]

...

tensor(21.2307)
tensor(21.3130)
tensor(20.9536)
current progress: 8/10, -0.010733


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(20.8755)
tensor(21.1942)
tensor(20.7004)
current progress: 9/10, -0.004208


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(19.6931)
tensor(19.5590)
tensor(19.3609)
current progress: 10/10, 0.003112
0.0004635102063348208


In [ ]:
# 0, 1, 2
mode = 2 
# 0, 1 ,2, 3
noise_level = 1


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(21.9757)
tensor(21.9429)
tensor(22.0556)
current progress: 1/10, -0.004023


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(23.3207)
tensor(21.9970)
tensor(23.8986)
current progress: 2/10, 0.038178


...

tensor(21.2443)
tensor(21.2424)
tensor(21.2875)
current progress: 8/10, -0.015142


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(20.7197)
tensor(21.4195)
tensor(20.6046)
current progress: 9/10, -0.003787


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(19.3029)
tensor(19.4214)
tensor(19.3605)
current progress: 10/10, 0.012097
0.004463120719103582


In [ ]:
# 0, 1, 2
mode = 2
# 0, 1 ,2, 3
noise_level = 2


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

tensor(21.6525)
tensor(21.9123)
tensor(21.9612)
current progress: 1/10, 0.002799


  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(23.7518)
tensor(24.3993)
tensor(24.4232)
current progress: 2/10, -0.008483


...

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(20.4562)
tensor(20.4562)
tensor(20.4562)
current progress: 9/10, 0.018213


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(18.5365)
tensor(18.8602)
tensor(19.5775)
current progress: 10/10, 0.030987
0.004003731924739516


In [ ]:
# 0, 1, 2
mode = 2
# 0, 1 ,2, 3
noise_level = 3


curr_clipscore_batch = []

for i,gen_prompts in enumerate(re_prompts):
  caption = full_prompts[i]

  # load input image and input mask image
  input_img_filename = "input_" + str(i) +".jpg"
  input_mask_filename = "edit_mask_"+str(i)+"_mode"+str(mode)+"_noise"+ str(noise_level) +".jpg"
  input_img = PIL.Image.open(input_img_filename)
  mask_img = PIL.Image.open(input_mask_filename)

  # calculate generation output
  output_gen = inpainting(mask_img, input_img, gen_prompts)

  # clean variables
  del input_img_filename
  del input_mask_filename
  del input_img
  del mask_img

  # calculate clipscore
  curr_clipscore = calculate_clipscore(output_gen, caption)
  clipscore_change = (original_clipscore[i] - curr_clipscore)/original_clipscore[i]
  curr_clipscore_batch.append(clipscore_change)

  # print progress
  curr_i = i+1
  print("current progress: %d/%d, %f" % (curr_i, total_samples, clipscore_change))

  # clean variables
  del output_gen
  del clipscore_change
  del curr_i


# save data to file
curr_batch_avg = np.mean(curr_clipscore_batch)
del curr_clipscore_batch


print(curr_batch_avg)
del curr_batch_avg

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(21.7501)
tensor(21.9595)
tensor(21.9757)
current progress: 1/10, 0.000375


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(24.4003)
tensor(24.3800)
tensor(24.3936)
current progress: 2/10, -0.016815


...

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


tensor(20.4562)
tensor(20.3546)
tensor(20.6555)
current progress: 9/10, 0.016650


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(18.6359)
tensor(19.0130)
tensor(19.5773)
current progress: 10/10, 0.026700
-0.0001807260074849288
